#### Experiment 01: usim as the backing DES framework feasibility

In [3]:
from usim import time, run, Scope, Tracked
class Server:
    def __init__(self, counter, s2):
        self.counter = counter
        self.s2 = s2

    async def get_and_increment(self, request):
        print(f'{time.now} received call with request')
        await (time + 5)
        print(f'{time.now} returning')
        response = self.counter
        self.counter += 1
        await request.set(response)

    def resolve(self):
        return self.s2.get_and_increment, self.s2

    async def rpc(self):
        async with Scope() as client:
            request = Tracked(0)
            api, instance = self.resolve()
            await client.do(api(request))
            response = request.value
            print(f'{time.now} rpc {response=}')

In [4]:
s2 = Server(1000, None)
s1 = Server(0, s2)

async def call_server():
    print(f'{time.now} starting calls')
    await s1.rpc()

run(call_server())

0 starting calls
0 received call with request
5 returning
5 rpc response=1000


#### Experiment 02: Adding some structure to the code for upcoming framework design ideas
* dns resolution of hostnames to server can be simulated by a dictionary lookup to the actual server class
* latency can be simulated by await time
* I also want to simulate the overloading of CPU resources and system behavior of services under load as soon as possible. If some non-linearity of latency can be observed (by logging and plotting), then it indicates this simulation is successful and can be further developed.
  * Probably requires a request queue in the server and cpu resource modelling.

In [6]:
from usim import run, time, Scope, Tracked

class Server:
    def __init__(self):
        self.counter = 0

    async def increment_counter(self, payload):
        await (time + 1)
        self.counter += 1
        response_payload = {
            'counter': self.counter,
            'og_payload': payload
        }
        return Tracked(response_payload)

In [7]:
server_dict = {'a': Server()}

async def rpc(url: str, api: str, payload: dict):
    s = server_dict[url]
    a = getattr(s, api)
    response = await a(payload)
    return response

async def client_logic():
    for _ in range(10):
        print(f'{time.now} - calling rpc')
        response = await rpc('a', 'increment_counter', {1})
        print(f'{time.now} - {response.value}')

run(client_logic(), till=20)

0 - calling rpc
1 - {'counter': 1, 'og_payload': {1}}
1 - calling rpc
2 - {'counter': 2, 'og_payload': {1}}
2 - calling rpc
3 - {'counter': 3, 'og_payload': {1}}
3 - calling rpc
4 - {'counter': 4, 'og_payload': {1}}
4 - calling rpc
5 - {'counter': 5, 'og_payload': {1}}
5 - calling rpc
6 - {'counter': 6, 'og_payload': {1}}
6 - calling rpc
7 - {'counter': 7, 'og_payload': {1}}
7 - calling rpc
8 - {'counter': 8, 'og_payload': {1}}
8 - calling rpc
9 - {'counter': 9, 'og_payload': {1}}
9 - calling rpc
10 - {'counter': 10, 'og_payload': {1}}


#### Experiment 03: adding more layers with the same forumla. See if it scales

In [17]:
from usim import run, time, Tracked
from itertools import cycle

class Logger:
    @staticmethod
    def log(s):
        print(f'{time.now:0>5} - {s}')

class LoadBalancer:
    def __init__(self, server_names):
        self.counter = 0
        self.server_names = cycle(server_names)

    async def compute(self, payload):
        server_name = next(self.server_names)
        Logger.log(f'lb received request forwarding to >> {server_name}')
        await (time + 20)
        return await rpc(server_name, 'compute', payload)

class ComputeServer:
    def __init__(self, name):
        self.name = name
    
    async def compute(self, payload):
        Logger.log(f'{self.name} received {payload}')
        await (time + 200)
        a, b = payload['a'], payload['b']
        response = {
            'result': a + b
        }
        Logger.log(f'{self.name} response {response}')
        return Tracked(response)
    
cs1 = ComputeServer('cs1')
cs2 = ComputeServer('cs2')
cs3 = ComputeServer('cs3')
lb = LoadBalancer(['cs1', 'cs2', 'cs3'])

server_dict = {
    'ComputeService': lb,
    'cs1' : cs1,
    'cs2' : cs2,
    'cs3' : cs3
}

async def rpc(url: str, api: str, payload: dict):
    s = server_dict[url]
    a = getattr(s, api)
    response = await a(payload)
    return response

async def client_logic():
    for _ in range(10):
        Logger.log(f'client calling rpc')
        response = await rpc('ComputeService', 'compute', {'a': 4, 'b': 2})
        Logger.log(f'client response {response.value}')

run(client_logic(), till=1000)

00000 - client calling rpc
00000 - lb received request forwarding to >> cs1
00020 - cs1 received {'a': 4, 'b': 2}
00220 - cs1 response {'result': 6}
00220 - client response {'result': 6}
00220 - client calling rpc
00220 - lb received request forwarding to >> cs2
00240 - cs2 received {'a': 4, 'b': 2}
00440 - cs2 response {'result': 6}
00440 - client response {'result': 6}
00440 - client calling rpc
00440 - lb received request forwarding to >> cs3
00460 - cs3 received {'a': 4, 'b': 2}
00660 - cs3 response {'result': 6}
00660 - client response {'result': 6}
00660 - client calling rpc
00660 - lb received request forwarding to >> cs1
00680 - cs1 received {'a': 4, 'b': 2}
00880 - cs1 response {'result': 6}
00880 - client response {'result': 6}
00880 - client calling rpc
00880 - lb received request forwarding to >> cs2
00900 - cs2 received {'a': 4, 'b': 2}
